In [1]:
import os
# 【硬性要求】加载环境变量 (自动读取本地 .env 文件中的 OPENAI_API_KEY 等)
from dotenv import load_dotenv
load_dotenv(override=True)

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough

# ==========================================
# 1. 实例化 LLMs (展示老师提到的"挑选最好的LLM"思想)
# ==========================================
# 便宜快捷的模型，用于简单的前置/后置任务 (如生成标题、大纲、摘要)
llm_fast = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)
# 强大但稍贵的模型，用于核心的重度创作任务 (如撰写长篇博客)
llm_smart = ChatOpenAI(model="gpt-4o", temperature=0.7) 

# ==========================================
# 2. 构建子链 (Sub-chains)
# ==========================================

# 环节 A: 标题链
title_chain = (
    ChatPromptTemplate.from_template("Generate an impactful title for {input}")
    | llm_fast
    | StrOutputParser()  # 此时输出纯文本，例如 "The Future of AI Jobs"
    # 【保姆级注释】: 这里将纯文本组装成字典 {"title": "The Future of AI Jobs"}
    # 以便下一个环节的 Prompt 能够识别 {title} 这个变量。
    | {"title": RunnablePassthrough()} 
)

# 环节 B: 大纲链
outline_chain = (
    ChatPromptTemplate.from_template("Generate a detailed outline for {title}")
    | llm_fast
    | StrOutputParser()
    | {"outline": RunnablePassthrough()} # 将输出组装为 {"outline": "1. Intro..."}
)

# 环节 C: 博客正文链 (调用更强大的 LLM)
blog_chain = (
    ChatPromptTemplate.from_template("Generate a 200 word blog post based on the outline: {outline}")
    | llm_smart  # 核心创作，切换至 GPT-4o
    | StrOutputParser()
    | {"blog": RunnablePassthrough()} # 将输出组装为 {"blog": "In recent years..."}
)

# 环节 D: 摘要链
summary_chain = (
    ChatPromptTemplate.from_template("Generate a summary for the post: {blog}")
    | llm_fast
    | StrOutputParser()
    # 【保姆级注释】: 最后一个链，不需要再向后传递特定的 key，直接输出字符串结果给用户即可，所以不需要 RunnablePassthrough 封装。
)

# ==========================================
# 3. 组装超级链 (Chain of Chains)
# ==========================================
# 通过 | 符号，像拼乐高一样把所有环节串联起来
content_chain = title_chain | outline_chain | blog_chain | summary_chain

# ==========================================
# 4. 执行测试
# ==========================================
if __name__ == "__main__":
    print("开始执行内容生成流水线，这可能需要几十秒的时间，请稍候...\n")
    # 输入最初始的变量 {"input": ...}
    final_summary = content_chain.invoke({"input": "The impact of AI on jobs"})
    
    print("=== 最终生成的社交媒体摘要 ===")
    print(final_summary)

开始执行内容生成流水线，这可能需要几十秒的时间，请稍候...

=== 最终生成的社交媒体摘要 ===
The AI revolution is reshaping the employment landscape by enhancing efficiency but also posing challenges such as job displacement. While routine jobs are at risk of automation, new opportunities in AI development and data analysis are emerging. Education and training programs are essential for preparing the workforce for these changes. The future of work may involve a freelance economy, requiring continuous learning and upskilling. Governments must regulate AI deployment and support displaced workers. Adapting to the AI revolution requires lifelong learning and strategic planning to harness AI's benefits while mitigating its challenges.
